In [1]:
import pandas as pd
import numpy as np

In [4]:
import json

with open('./hackthon data/mcc_codes.json', 'r') as f:
    codes = json.load(f)

In [10]:
with open('./hackthon data/train_fraud_labels.json', 'r') as f:
    labels = json.load(f)

In [11]:
labels = labels['target']

In [24]:
targets = pd.DataFrame(list(labels.items()), columns=['transaction_id', 'target'])

targets.head()

,transaction_id,target
0,19765990,No
1,22160255,No
2,17566794,No
3,17318690,No
4,20994060,No


In [25]:
targets['target'] = targets['target'].replace({'No': 0, 'Yes': 1})
targets.head()

C:\Users\ayman\AppData\Local\Temp\ipykernel_17540\571482086.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  targets['target'] = targets['target'].replace({'No': 0, 'Yes': 1})


,transaction_id,target
0,19765990,0
1,22160255,0
2,17566794,0
3,17318690,0
4,20994060,0


In [29]:
# Check for NaN values in the targets DataFrame
nan_counts = targets.isna().sum()
print(nan_counts)

transaction_id    0
target            0
dtype: int64


In [30]:
df = pd.read_csv('./hackthon data/transactions_train.csv')
df.head()

,transaction_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,19765990,2017-07-07 09:53:00,1581,2519,$12.35,Chip Transaction,20519,Arnold,MO,63010.0,5942,NaN
1,22160255,2018-11-26 17:44:00,1862,4049,$58.29,Chip Transaction,98648,Des Moines,IA,50313.0,5814,NaN
2,17566794,2016-03-26 12:42:00,1967,3367,$11.03,Chip Transaction,46978,Lake Forest,CA,92630.0,5411,NaN
3,17318690,2016-02-01 08:30:00,921,3457,$85.74,Chip Transaction,63701,Rush,NY,14543.0,5411,NaN
4,20994060,2018-03-24 14:42:00,456,2800,$13.43,Chip Transaction,83271,Estero,FL,33928.0,4214,NaN


In [34]:
df['client_id'].nunique()

852

In [35]:
df['transaction_id'].nunique()

210000

In [33]:
# Ensure both columns are of the same type (string/object) before merging
df['transaction_id'] = df['transaction_id'].astype(str)
merged_df = df.merge(targets, left_on='transaction_id', right_on='transaction_id', how='left')
merged_df.head()

,transaction_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,target
0,19765990,2017-07-07 09:53:00,1581,2519,$12.35,Chip Transaction,20519,Arnold,MO,63010.0,5942,NaN,0
1,22160255,2018-11-26 17:44:00,1862,4049,$58.29,Chip Transaction,98648,Des Moines,IA,50313.0,5814,NaN,0
2,17566794,2016-03-26 12:42:00,1967,3367,$11.03,Chip Transaction,46978,Lake Forest,CA,92630.0,5411,NaN,0
3,17318690,2016-02-01 08:30:00,921,3457,$85.74,Chip Transaction,63701,Rush,NY,14543.0,5411,NaN,0
4,20994060,2018-03-24 14:42:00,456,2800,$13.43,Chip Transaction,83271,Estero,FL,33928.0,4214,NaN,0


In [37]:
cards = pd.read_csv('./hackthon data/cards_data.csv')
cards.head()

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No


In [41]:
idx_df = merged_df['client_id'].unique()

idx_df.shape

(852,)

In [42]:
cards = cards[cards['client_id'].isin(idx_df)]
cards.head()

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
5,4537,1746,Visa,Credit,4404898874682993,09/2003,736,YES,1,$27500,09/2003,2012,No
6,1278,1746,Visa,Debit,4001482973848631,07/2022,972,YES,2,$28508,02/2011,2011,No
7,3687,1746,Mastercard,Debit,5627220683410948,06/2022,48,YES,2,$9022,07/2003,2015,No
8,3465,1746,Mastercard,Debit (Prepaid),5711382187309326,11/2020,722,YES,2,$54,06/2010,2015,No
9,3754,1746,Mastercard,Debit (Prepaid),5766121508358701,02/2023,908,YES,1,$99,07/2006,2012,No


In [44]:
users = pd.read_csv('./hackthon data/users_data.csv')
users.head()

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1


In [49]:
idx_cards = cards['client_id'].unique()

users = users[users['id'].isin(idx_cards)]
users.head()

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
6,1075,36,67,1983,12,Female,5695 Fifth Street,38.22,-85.74,$25258,$51500,$102286,672,3
13,1094,34,62,1985,10,Male,74786 Jefferson Drive,44.75,-85.60,$20325,$41442,$78833,712,3
15,1660,41,68,1978,4,Female,40 Washington Drive,36.73,-102.51,$11342,$23123,$5079,723,6
19,511,66,68,1953,10,Male,153 Tenth Lane,32.21,-110.88,$17460,$35602,$55369,661,5


In [50]:
users = users.rename(columns={'id': 'client_id'})
users.head()

,client_id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
6,1075,36,67,1983,12,Female,5695 Fifth Street,38.22,-85.74,$25258,$51500,$102286,672,3
13,1094,34,62,1985,10,Male,74786 Jefferson Drive,44.75,-85.60,$20325,$41442,$78833,712,3
15,1660,41,68,1978,4,Female,40 Washington Drive,36.73,-102.51,$11342,$23123,$5079,723,6
19,511,66,68,1953,10,Male,153 Tenth Lane,32.21,-110.88,$17460,$35602,$55369,661,5


In [51]:
users.drop(columns=['address'], inplace=True)

users_cards = users.merge(cards, on='client_id', how='inner')
users_cards.head()

,client_id,current_age,retirement_age,birth_year,birth_month,gender,latitude,longitude,per_capita_income,yearly_income,...,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,1746,53,68,1966,12,Female,40.76,-73.74,$37891,$77254,...,Credit,4404898874682993,09/2003,736,YES,1,$27500,09/2003,2012,No
1,1746,53,68,1966,12,Female,40.76,-73.74,$37891,$77254,...,Debit,4001482973848631,07/2022,972,YES,2,$28508,02/2011,2011,No
2,1746,53,68,1966,12,Female,40.76,-73.74,$37891,$77254,...,Debit,5627220683410948,06/2022,48,YES,2,$9022,07/2003,2015,No
3,1746,53,68,1966,12,Female,40.76,-73.74,$37891,$77254,...,Debit (Prepaid),5711382187309326,11/2020,722,YES,2,$54,06/2010,2015,No
4,1746,53,68,1966,12,Female,40.76,-73.74,$37891,$77254,...,Debit (Prepaid),5766121508358701,02/2023,908,YES,1,$99,07/2006,2012,No


In [57]:
merged_df.shape

(210000, 13)

In [52]:
final_df = merged_df.merge(users_cards, on='client_id', how='inner')
final_df.head()

,transaction_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,...,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,19765990,2017-07-07 09:53:00,1581,2519,$12.35,Chip Transaction,20519,Arnold,MO,63010.0,...,Credit,335921476257243,02/2022,743,NO,1,$11300,06/2003,2007,No
1,19765990,2017-07-07 09:53:00,1581,2519,$12.35,Chip Transaction,20519,Arnold,MO,63010.0,...,Debit,4564719638698318,12/2020,446,YES,1,$26596,04/2006,2015,No
2,19765990,2017-07-07 09:53:00,1581,2519,$12.35,Chip Transaction,20519,Arnold,MO,63010.0,...,Debit,5862160658521395,01/2023,117,YES,2,$1807,11/2008,2013,No
3,19765990,2017-07-07 09:53:00,1581,2519,$12.35,Chip Transaction,20519,Arnold,MO,63010.0,...,Credit,6494282265479617,07/2024,471,YES,1,$9700,11/2002,2005,No
4,19765990,2017-07-07 09:53:00,1581,2519,$12.35,Chip Transaction,20519,Arnold,MO,63010.0,...,Credit,5607170404260670,03/2020,51,YES,1,$12300,12/2005,2010,No


In [55]:
final_df.isna().sum()

transaction_id                0
date                          0
client_id                     0
card_id                       0
amount                        0
use_chip                      0
merchant_id                   0
merchant_city                 0
merchant_state            94495
zip                      100072
mcc                           0
errors                   791615
target                        0
current_age                   0
retirement_age                0
birth_year                    0
birth_month                   0
gender                        0
latitude                      0
longitude                     0
per_capita_income             0
yearly_income                 0
total_debt                    0
credit_score                  0
num_credit_cards              0
id                            0
card_brand                    0
card_type                     0
card_number                   0
expires                       0
cvv                           0
has_chip

In [56]:
final_df.shape

(804141, 37)

In [58]:
final_df = merged_df.merge(users_cards, left_on=['client_id', 'card_id'], right_on=['client_id', 'id'], how='inner')
final_df.head()

,transaction_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,...,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,19765990,2017-07-07 09:53:00,1581,2519,$12.35,Chip Transaction,20519,Arnold,MO,63010.0,...,Debit,4564719638698318,12/2020,446,YES,1,$26596,04/2006,2015,No
1,22160255,2018-11-26 17:44:00,1862,4049,$58.29,Chip Transaction,98648,Des Moines,IA,50313.0,...,Credit,4495571717001479,09/2023,778,YES,2,$10200,07/2016,2016,No
2,17566794,2016-03-26 12:42:00,1967,3367,$11.03,Chip Transaction,46978,Lake Forest,CA,92630.0,...,Debit,4738213924782743,09/2020,383,YES,2,$38120,06/2007,2011,No
3,17318690,2016-02-01 08:30:00,921,3457,$85.74,Chip Transaction,63701,Rush,NY,14543.0,...,Debit,4319734914723878,05/2020,720,YES,1,$35073,06/2009,2009,No
4,20994060,2018-03-24 14:42:00,456,2800,$13.43,Chip Transaction,83271,Estero,FL,33928.0,...,Credit,4751933848032848,12/2024,614,YES,1,$5800,05/1998,2009,No


In [59]:
final_df.shape

(210000, 37)